# **RAMP starting kit on the French housing regression dataset**

## Table of Contents
* [Introduction](#introduction)
* [The dataset](#dataset)
* [Requirements](#requirements)
* [Data exploration](#exploration)
* [Base model](#base_model)
* [Submitting on RAMP](#submitting)

# Introduction

# The dataset

For this challenge, the data were first preprocessed and then split in order to preserve a private test set on which to evaluate the models on our servers. This leaves 1 587 807 observations in the public train set and 396 952 observations in the public test set. 

## Requirements

In [218]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

## Data exploration

In [219]:
train = pd.read_csv('data/train_public.csv')
train

,Date mutation,Nature mutation,Valeur fonciere,Code departement,Code commune,Type local,Nombre pieces principales,Nature culture,Surface terrain,Surface reelle bati
0,10/09/2021,Vente,"874075,00",13,50,Dépendance,0.0,T,3000.0,0.0
1,20/07/2021,Vente,"149064,00",23,261,NaN,NaN,T,65635.0,NaN
2,12/05/2021,Vente,"306000,00",60,139,Maison,4.0,S,1049.0,70.0
3,09/02/2021,Vente,"560000,00",75,113,Appartement,2.0,NaN,NaN,48.0
4,12/05/2021,Vente,"175000,00",6,29,Dépendance,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
1587802,11/01/2021,Vente,"153000,00",77,90,Maison,4.0,S,131.0,103.0
1587803,07/04/2021,Vente,"10000,00",17,202,NaN,NaN,S,3.0,NaN
1587804,26/01/2021,Vente,"170130,00",65,440,Appartement,4.0,NaN,NaN,96.0
1587805,29/09/2021,Vente,"27188,00",19,31,NaN,NaN,S,532.0,NaN


In [220]:
test = pd.read_csv('data/test_public.csv')
test

,Date mutation,Nature mutation,Valeur fonciere,Code departement,Code commune,Type local,Nombre pieces principales,Nature culture,Surface terrain,Surface reelle bati
0,11/02/2021,Vente,"197000,00",33,522,Appartement,3.0,NaN,NaN,64.0
1,15/10/2021,Vente,"570120,00",14,715,Appartement,1.0,S,104.0,30.0
2,24/08/2021,Vente,"380000,00",74,263,Maison,4.0,S,639.0,76.0
3,08/01/2021,Vente,"11900,00",63,163,NaN,NaN,P,17130.0,NaN
4,13/04/2021,Vente,"165900,00",37,158,Dépendance,0.0,S,606.0,0.0
...,...,...,...,...,...,...,...,...,...,...
396947,10/09/2021,Vente,"164000,00",16,393,Dépendance,0.0,S,665.0,0.0
396948,16/12/2021,Vente,"98000,00",34,324,Appartement,1.0,NaN,NaN,19.0
396949,01/12/2021,Vente,"3000,00",64,161,NaN,NaN,BT,8250.0,NaN
396950,05/08/2021,Vente,"358800,00",2B,37,NaN,NaN,B,1102.0,NaN


In [221]:
train.columns

Index(['Date mutation', 'Nature mutation', 'Valeur fonciere',
       'Code departement', 'Code commune', 'Type local',
       'Nombre pieces principales', 'Nature culture', 'Surface terrain',
       'Surface reelle bati'],
      dtype='object')

## Labels

In [222]:
train['Valeur fonciere'].unique().shape, train['Valeur fonciere'].unique()


((116890,),
 array(['874075,00', '149064,00', '306000,00', ..., '209245,00',
        '784090,00', '170130,00'], dtype=object))

In [223]:
train['Valeur fonciere'].value_counts(dropna=False)

Valeur fonciere
150000,00     13652
120000,00     12662
100000,00     12599
80000,00      11644
200000,00     11578
              ...  
613,50            1
205999,00         1
190740,00         1
1077950,00        1
170130,00         1
Name: count, Length: 116890, dtype: int64

First, we deal with tricky data types such as dates, and we create useful columns such as price per m2.

In [224]:
def to_float(x):
  try : 
    return float(x)
  except :
    return float(x.replace(',', '.'))
def to_string(x):
  try : return str(x)
  except : return x
def transform_dataset(df):
    df['Date mutation'] = pd.to_datetime( df['Date mutation'], format='%d/%m/%Y')
    df['Year'] = df['Date mutation'].dt.year
    df['Month'] = df['Date mutation'].dt.month
    df['Day'] = df['Date mutation'].dt.day
    df = df.drop(columns=['Date mutation', 'Nature culture', 'Nature mutation'])
    df['Valeur fonciere'] = df['Valeur fonciere'].apply(lambda x : to_float(x))
    df['Code departement'] = df['Code departement'].apply(lambda x : to_string(x))
    df['Code commune'] = df['Code commune'].apply(lambda x : to_string(x))
    df['Nombre pieces principales'] = df['Nombre pieces principales'].fillna(0)
    df['Nombre pieces principales'] = df['Nombre pieces principales'].apply(lambda x : int(x))
    df['Surface terrain'] = df['Surface terrain'].apply(lambda x : float(x))
    df['Surface reelle bati'] = df['Surface reelle bati'].apply(lambda x : float(x))
    df.drop(df[df['Surface terrain'] <= 9].index, inplace = True)
    df.drop(df[df['Valeur fonciere'] <= 10].index, inplace = True)
    df['Prix m2'] = df['Valeur fonciere']/df['Surface terrain']
    df['Proportion terrain bati'] = df['Surface reelle bati']/df['Surface terrain']
    df.drop(df[df['Prix m2'] <= 0.1].index, inplace = True)
    df.drop_duplicates(subset=['Valeur fonciere', 'Code departement'], inplace=True)
    return df

In [225]:
train = transform_dataset(train)
test = transform_dataset(test)

Then, we inspect NaN values and deal with them.

In [226]:
train.isna().sum()

Valeur fonciere                  97
Code departement                  0
Code commune                      0
Type local                   121450
Nombre pieces principales         0
Surface terrain              107262
Surface reelle bati          121675
Year                              0
Month                             0
Day                               0
Prix m2                      107357
Proportion terrain bati      216967
dtype: int64

In [227]:
test.isna().sum()

Valeur fonciere                 94
Code departement                 0
Code commune                     0
Type local                   57542
Nombre pieces principales        0
Surface terrain              42576
Surface reelle bati          57626
Year                             0
Month                            0
Day                              0
Prix m2                      42666
Proportion terrain bati      96099
dtype: int64

We choose to drop them, as we will still have a sufficient amount of data for our task

In [228]:
train.dropna(inplace=True)
test.dropna(inplace=True)

Let's now deal with the categorical data.

In [229]:
train.dtypes

Valeur fonciere              float64
Code departement              object
Code commune                  object
Type local                    object
Nombre pieces principales      int64
Surface terrain              float64
Surface reelle bati          float64
Year                           int32
Month                          int32
Day                            int32
Prix m2                      float64
Proportion terrain bati      float64
dtype: object

In [230]:
train['Code departement'].unique()

array(['13', '60', '44', '59', '49', '37', '72', '14', '87', '33', '28',
       '85', '40', '95', '89', '17', '11', '93', '77', '91', '52', '6',
       '21', '24', '73', '80', '45', '27', '84', '83', '2', '1', '62',
       '63', '30', '23', '76', '29', '42', '7', '69', '92', '64', '31',
       '56', '74', '34', '78', '3', '82', '53', '79', '9', '50', '54',
       '71', '19', '35', '47', '58', '55', '38', '26', '12', '66', '51',
       '65', '974', '39', '32', '94', '16', '86', '81', '10', '22', '15',
       '88', '36', '90', '43', '41', '2B', '8', '61', '4', '25', '75',
       '70', '971', '46', '972', '18', '973', '48', '5', '2A'],
      dtype=object)

In [231]:
train['Code departement'] = train['Code departement'].replace({'2A': '2', '2B': '2'})
test['Code departement'] = test['Code departement'].replace({'2A': '2', '2B': '2'})

In [232]:
train['Code commune'] = pd.to_numeric(train['Code commune'], errors='coerce')
test['Code commune'] = pd.to_numeric(test['Code commune'], errors='coerce')

In [233]:
train = pd.get_dummies(train, columns=['Type local'])
test = pd.get_dummies(test, columns=['Type local'])

In [234]:
test.columns

Index(['Valeur fonciere', 'Code departement', 'Code commune',
       'Nombre pieces principales', 'Surface terrain', 'Surface reelle bati',
       'Year', 'Month', 'Day', 'Prix m2', 'Proportion terrain bati',
       'Type local_Appartement', 'Type local_Dépendance',
       'Type local_Local industriel. commercial ou assimilé',
       'Type local_Maison'],
      dtype='object')

Finally, we separate the labels from the features for both datasets

In [235]:
X_train = train.drop(columns=['Valeur fonciere'])
X_test = test.drop(columns=['Valeur fonciere'])
y_train = train['Valeur fonciere']
y_test = test['Valeur fonciere']

In [236]:
X_test.columns

Index(['Code departement', 'Code commune', 'Nombre pieces principales',
       'Surface terrain', 'Surface reelle bati', 'Year', 'Month', 'Day',
       'Prix m2', 'Proportion terrain bati', 'Type local_Appartement',
       'Type local_Dépendance',
       'Type local_Local industriel. commercial ou assimilé',
       'Type local_Maison'],
      dtype='object')

# Base model

In [237]:
clf = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])


In [238]:
clf.fit(X_train, y_train)
clf

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', LinearRegression())])

In [239]:
y_pred = clf.predict(X_test)

In [240]:
y_pred.shape

(45075,)

In [241]:
y_test.shape

(45075,)

In [242]:
accuracy_score(y_test, y_pred)

ValueError: continuous is not supported